In [1]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hoduy\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


 # [Genetic Algorithm - GA](https://en.wikipedia.org/wiki/Genetic_algorithm)  

Genetic algorithms (GAs) are search and optimization techniques inspired by natural selection and genetics. GAs are used to solve complex problems by simulating biological mechanisms such as mutation, crossover, and selection.

## 1. Basic Principle

GA works on Charles Darwin's principle of natural selection: **"Better adapted individuals will have more chances to survive and reproduce."**. An optimization problem in GA is usually represented through the following steps:

1. **Population Initialization**: Create an initial set of individuals (called a population).
2. **Evaluation**: Determine the fitness of each individual in the population.
3. **Selection**: Select the best individuals based on fitness values ​​to reproduce the next generation.
4. **Crossover**: Combine the genetic information of individuals to create new individuals.
5. **Mutation**: Randomly change a part of the genetic information of an individual to increase diversity.
6. **Iteration**: Repeat the above process until the stopping condition is reached (e.g., reaching the desired number of generations or the optimal value).

## 2. Components of GA

- **Individual**: A potential solution to the problem.

- **Population**: The set of current individuals.

- **Fitness**: A measure to evaluate the goodness of an individual.

- **Crossover**: A method of combining two parent individuals to create an offspring.

- **Mutation**: Randomly changing a part of the individual's genes to create a difference.

## 3. Algorithm process

1. **Initial population**:
- Randomly generate initial individuals.

2. **Fitness assessment**:
- Calculate the fitness value of each individual in the population.

3. **Selection**:
- Select the best individuals to form the next generation.
4. **Crossover**:
- Select two parent individuals and combine their genes to create an offspring.

5. **Mutation**:
- Randomly change one or more genes of the offspring to increase diversity.

6. **Population Update**:
- Replace old individuals with new individuals.

7. **Stopping Condition Check**:
- Stop if the condition is met (e.g. maximum number of generations or desired fitness).

## 4. Applications of GA

- Optimization in many fields such as:
- Traveling Salesman Problem (TSP).
- Scheduling.
- Design Optimization.
- Machine Learning: Optimizing network parameters and structures.

- Other research problems in biology, finance, and technology.

## 5. Advantages and disadvantages of GA

### Advantages:
- Can solve complex, non-linear problems.

- Search in large spaces with high efficiency.

- Does not require much information about the problem, only needs a fitness function.

### Disadvantages:
- Easy to fall into local extremes.

- Long computation time if not optimized.

Need to fine-tune parameters (population size, mutation rate, crossover rate).

# Implementing GA to solve the timetable problem

## 1. Timetable representation
- **Chromosome**: A feasible timetable.

- Gene = (Subject, Teacher, Class, Time, Classroom).

- **Instructor's free time**: Added to check the validity of the gene.

- **Hard** and **Soft** constraints:
- Hard constraint: Required to comply.

- Soft constraint: Desired, if violated, only fitness points will be reduced.

## 2. Hard Constraints
1. **Teachers do not teach at the same time**:
- `if Teacher_A teaches at T1 -> Teacher_A does not teach in another class at T1`.
2. **Classrooms do not teach at the same time**:
- `if Room_B is used at T2 -> Room_B is not used by another class at T2`.
3. **Subject assigned to the correct teacher**:

- `if Teacher_X is not qualified -> Cannot teach Subject_Y`.

4. **Each class only studies 1 subject at a time**:

- `if Class_C studies Subject_Z at T3 -> Cannot study other subjects at T3`.

5. **Maximum number of lessons per day**:

- `if Number of lessons > Limit -> Violation`.

## 3. Soft Constraints

1. **Lecturer's free time**:

- `if Time_T is not Teacher_A's free time -> Violation`.

# Simulate the timetable problem space

## 4. Problem space
- **3D timetable model**:
- **X-axis (Day)**: Days of the week (Monday → Saturday).

- **Y-axis (Period)**: Classes of the day (Period 1 → Period 8).

- **Z-axis (Classroom)**: Classrooms (A101, A102, A103...).

### For example:
- In the cell `(Saturday, Period 4, Room A101)`:
- The class card 86 represents the class coded **86**.

This corresponds to: "Saturday, Period 4, in room A101".

## 5. Overview workflow
### **Input**:
- List of subjects, teachers, classes, classrooms.
- Lecturer's free time.
- Hard and soft constraints (predefined).

### **Workflow**:
1. **Spatial modeling**:
- Create a 3D structure (Day, Period, Room) for the timetable.

2. **Generate initial population**:
- Randomly allocate subjects, teachers, classes into cells, obeying hard constraints.

3. **Optimize**:
- Repeat optimization steps (GA or other algorithm) to reduce the number of constraint violations and optimize soft constraints.

4. **Result**:
- Output the optimal timetable in 3D space.

## 3. Result
- The final timetable is represented in 3D space:

- **X-axis**: Day.

- **Y-axis**: Class.

- **Z-axis**: Classroom.

- Each cell contains subject, teacher, and class information.

In [2]:
import openpyxl
print(openpyxl.__version__)

3.1.5


In [3]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read excel

In [4]:

file_path = "ysc_2024.xlsx" 
sheets = pd.read_excel(file_path, sheet_name=None)
dfs = list(sheets.values())

In [5]:
df_subject = dfs[0]
df_teacher = dfs[1]
df_room = dfs[2]
df_freetime = dfs[3]

In [6]:
df_teacher.columns = df_teacher.columns.str.strip()


In [7]:
df_subject.head()

,STT,Mã môn học,Tên môn học,Hình thức,Mã học phần,Số tín chỉ,Số tiết,Chuyên ngành
0,1,3192,Kỹ năng làm việc nhóm,LT,4203003192,2,3,KHDL
1,2,3242,Giáo dục Quốc phòng và An Ninh 1,LT,4203003242,4,6,KHDL
2,3,3259,Toán cao cấp 1,LT,4203003259,2,6,KHDL
3,4,3848,Nhập môn Lập trình_TH,TH,4203003848,2,6,KHDL
4,5,941,Kỹ thuật lập trình,LT,4203000941,3,3,KHDL


In [8]:
df_teacher.head()

,Tên,Mã môn học,Khoa,Chuyên ngành,Title
0,Nguyễn Chí Kiên,3192,CNTT,KHDL,Tiến sĩ
1,Huỳnh Trung Hiếu,3242,CNTT,KHDL,PGS.TS
2,Lê Trọng Ngọc,3259,CNTT,KHDL,Tiến sĩ
3,Nguyễn Hữu Tình,2137,CNTT,KHDL,Thạc sĩ
4,Bùi Thanh Hùng,"3848, 941",CNTT,KHDL,Tiến sĩ


In [9]:
df_room.head()

,Tên Phòng,Hình thức
0,A1.01A,LT
1,A1.01B,TH
2,A1.02,LT
3,A1.03,LT
4,A1.04,TH


In [10]:
df_freetime.head()

,Nguyễn Chí Kiên,Lê Trọng Ngọc,Nguyễn Hữu Tình
0,"0, 0","2, 2","0, 1"
1,"1, 1","3, 3","2, 0"
2,"3, 2","4, 5","4, 4"


# Data processing conversion

### Subjects

In [11]:
subjects = [
    {
        'Mã môn học': f"{str(row['Mã môn học']).lstrip('0')}_{row['Hình thức']}",
        'Tên môn học': row['Tên môn học'],
        'Hình thức': row['Hình thức'],
        'Số tiết': row['Số tiết']
    }
    for _, row in df_subject.iterrows()
]
subjects

[{'Mã môn học': '3192_LT',
  'Tên môn học': 'Kỹ năng làm việc nhóm',
  'Hình thức': 'LT',
  'Số tiết': 3},
 {'Mã môn học': '3242_LT',
  'Tên môn học': 'Giáo dục Quốc phòng và An Ninh 1',
  'Hình thức': 'LT',
  'Số tiết': 6},
 {'Mã môn học': '3259_LT',
  'Tên môn học': 'Toán cao cấp 1',
  'Hình thức': 'LT',
  'Số tiết': 6},
 {'Mã môn học': '3848_TH',
  'Tên môn học': 'Nhập môn Lập trình_TH',
  'Hình thức': 'TH',
  'Số tiết': 6},
 {'Mã môn học': '941_LT',
  'Tên môn học': 'Kỹ thuật lập trình',
  'Hình thức': 'LT',
  'Số tiết': 3},
 {'Mã môn học': '2137_LT',
  'Tên môn học': 'Hệ thống Máy tính',
  'Hình thức': 'LT',
  'Số tiết': 3},
 {'Mã môn học': '941_TH',
  'Tên môn học': 'Kỹ thuật lập trình_TH',
  'Hình thức': 'TH',
  'Số tiết': 3},
 {'Mã môn học': '2137_TH',
  'Tên môn học': 'Hệ thống Máy tính_TH',
  'Hình thức': 'TH',
  'Số tiết': 3},
 {'Mã môn học': '3848_LT',
  'Tên môn học': 'Nhập môn Lập trình',
  'Hình thức': 'LT',
  'Số tiết': 3}]

In [12]:
for subject in subjects:
    print(subject['Số tiết'])

3
6
6
6
3
3
3
3
3


### Teachers

In [13]:
df_teacher['Mã môn học'] = df_teacher['Mã môn học'].astype(str)
df_subject['Mã môn học'] = df_subject['Mã môn học'].astype(str)

In [14]:
# df_teacher['Mã môn học'], df_subject['Mã môn học'] 
df_teacher_expanded = df_teacher.assign(
    Mã_môn_học_tách=df_teacher['Mã môn học'].str.split(", ")
).explode('Mã_môn_học_tách')

In [15]:
# Đảm bảo không có giá trị NaN trong cột 'Mã_môn_học_tách'
df_teacher_expanded.dropna(subset=['Mã_môn_học_tách'], inplace=True)

In [16]:
df_teacher_expanded = df_teacher_expanded.merge(
    df_subject[['Mã môn học', 'Hình thức']].rename(columns={'Mã môn học': 'Mã_môn_học_tách'}),
    on='Mã_môn_học_tách',
    how='left'
)

In [17]:
df_teacher_expanded['Mã_môn_học_Hình_thức'] = df_teacher_expanded['Mã_môn_học_tách'].str.lstrip("0") + "_" + df_teacher_expanded['Hình thức']

In [18]:
teacher_subjects = {
    teacher: df_teacher_expanded[df_teacher_expanded['Tên'] == teacher]['Mã_môn_học_Hình_thức'].dropna().tolist()
    for teacher in df_teacher['Tên']
}

### Room

In [19]:
rooms = df_room.to_dict(orient='records')
rooms

[{'Tên Phòng': 'A1.01A', 'Hình thức': 'LT'},
 {'Tên Phòng': 'A1.01B', 'Hình thức': 'TH'},
 {'Tên Phòng': 'A1.02', 'Hình thức': 'LT'},
 {'Tên Phòng': 'A1.03', 'Hình thức': 'LT'},
 {'Tên Phòng': 'A1.04', 'Hình thức': 'TH'},
 {'Tên Phòng': 'A1.05', 'Hình thức': 'LT'},
 {'Tên Phòng': 'A2.01', 'Hình thức': 'TH'},
 {'Tên Phòng': 'A2.02', 'Hình thức': 'LT'},
 {'Tên Phòng': 'A2.03', 'Hình thức': 'TH'},
 {'Tên Phòng': 'A2.04', 'Hình thức': 'TH'}]

### Teachers - Freetime

In [20]:
free_time = {
    col: [tuple(map(int, cell.split(", "))) for cell in df_freetime[col].dropna()]
    for col in df_freetime.columns
}
free_time

{'Nguyễn Chí Kiên': [(0, 0), (1, 1), (3, 2)],
 'Lê Trọng Ngọc': [(2, 2), (3, 3), (4, 5)],
 'Nguyễn Hữu Tình': [(0, 1), (2, 0), (4, 4)]}

In [21]:
NUM_PERIODS = 5   
NUM_DAYS = 7    # Số ngày trong tuần (Thứ 2 - CN)
NUM_WEEKS = 1    
NUM_ROOMS = len(rooms)  
POPULATION_SIZE = 1000
NUM_GENERATIONS = 10000
MUTATION_RATE = 0.1

# Generate chromosome

In [22]:
def assign_periods(chromosome, subject, teacher, subject_code, subject_name, subject_form, subject_periods, assigned_days):
    periods_assigned = 0
    attempts = 0

    while periods_assigned < subject_periods and attempts < 300:
        week = random.randint(0, NUM_WEEKS - 1)
        day = random.randint(0, NUM_DAYS - 1)
        period = random.randint(0, NUM_PERIODS - 1)
        room_idx = random.randint(0, NUM_ROOMS - 1)

        room = rooms[room_idx]
        if room['Hình thức'] != subject_form:
            attempts += 1
            continue

        # Kiểm tra số buổi học
        if subject_periods == 6 and len(assigned_days) >= 2 and day not in assigned_days:
            attempts += 1
            continue

        if chromosome[period, day, week, room_idx] is None:
            chromosome[period, day, week, room_idx] = f"{subject_code},{subject_name},{teacher}"
            periods_assigned += 1
            assigned_days.add(day)
            # Nếu đã gán đủ 2 ngày cho môn học có số tiết 6, không cho thêm ngày mới
            if subject_periods == 6 and len(assigned_days) > 2:
                periods_assigned -= 1
                chromosome[period, day, week, room_idx] = None
                attempts += 1
    return periods_assigned, attempts

In [23]:
def generate_chromosome():
    # Khởi tạo nhiễm sắc thể với cấu trúc 3D (Periods x Days x Weeks x Rooms)
    chromosome = np.full((NUM_PERIODS, NUM_DAYS, NUM_WEEKS, NUM_ROOMS), None, dtype=object)

    for subject in subjects:
        subject_code = str(subject['Mã môn học'])
        subject_name = subject['Tên môn học']
        subject_form = subject['Hình thức']
        subject_periods = subject['Số tiết']
        teacher_candidates = [t for t, subjs in teacher_subjects.items() if subject_code in subjs]

        if not teacher_candidates:
            continue

        teacher = random.choice(teacher_candidates)
        assigned_days = set()

        # gán buổi học
        periods_assigned, attempts = assign_periods(
            chromosome, subject, teacher, subject_code, subject_name, subject_form, subject_periods, assigned_days
        )
    return chromosome

# Fitness

In [24]:
def calculate_teacher_violations(chromosome, period, day, week, teacher_set, room_idx, entry):
    violations = 0
    subject_code, subject_name, teacher = entry.split(',')
    # gian vien khong day hai mon cung mot thoi diem
    if teacher in teacher_set:
        violations += 10
    teacher_set.add(teacher)
    return violations, teacher_set

In [25]:
def check_room_form_violations(entry, room):
    violations = 0
    subject_code, subject_name, teacher = entry.split(',')
    subject_form = [s['Hình thức'] for s in subjects if str(s['Mã môn học']) == subject_code][0]

    # Kiểm tra hình thức phòng học
    if room['Hình thức'] != subject_form:
        violations += 5
    return violations

In [26]:
def update_subject_periods(entry, subject_count, subject_periods, period, day):
    subject_code, subject_name, teacher = entry.split(',')

    # Đếm số lần môn học được dạy
    if subject_code in subject_count:
        subject_count[subject_code] += 1
        # Lưu ca học
        subject_periods[subject_code].add((period, day)) 
    else:
        subject_count[subject_code] = 1
        subject_periods[subject_code] = {(period, day)}

    return subject_count, subject_periods

In [27]:
def calculate_schedule_violations(subject_count, subject_periods):
    violations = 0

    # mỗi môn học có ít nhất một buổi
    # Phạt nặng nếu môn không được dạy
    for subject_code, count in subject_count.items():
        if count < 1:
            violations += 20  

    # Kiểm tra trên số ca học
    for subject in subjects:
        subject_code = str(subject['Mã môn học'])
        num_periods = len(subject_periods[subject_code])

        # Phạt nặng môn học 6 tiết > 2 ca
        if subject['Số tiết'] == 6 and num_periods > 2:
            # pha tren moi ca
            violations += 30 * (num_periods - 2)

        # Phạt nặng môn học 3 tiết > 1 ca
        if subject['Số tiết'] == 3 and num_periods > 1:
            # pha tren moi ca
            violations += 30 * (num_periods - 1)  
    return violations

In [28]:
def fitness(chromosome):
    violations = 0
    # Đảm bảo tất cả mã môn học được chuyển thành chuỗi
    subject_count = {str(subject['Mã môn học']): 0 for subject in subjects}  # Đếm số buổi học cho từng môn
    subject_periods = {str(subject['Mã môn học']): set() for subject in subjects}  # Theo dõi số ca học của từng môn

    for week in range(NUM_WEEKS):
        for day in range(NUM_DAYS):
            for period in range(NUM_PERIODS):
                teacher_set = set()

                for room_idx in range(NUM_ROOMS):
                    entry = chromosome[period, day, week, room_idx]
                    if entry:
                        # Tính vi phạm đến giáo viên
                        teacher_violations, teacher_set = calculate_teacher_violations(
                            chromosome, period, day, week, teacher_set, room_idx, entry
                        )
                        violations += teacher_violations

                        # Tính vi phạm hình thức phòng học
                        room = rooms[room_idx]
                        violations += check_room_form_violations(entry, room)

                        # Cập nhật thông tin về số lần dạy và ca học
                        subject_count, subject_periods = update_subject_periods(
                            entry, subject_count, subject_periods, period, day
                        )

    # Tính vi phạm
    violations += calculate_schedule_violations(subject_count, subject_periods)
    return 1 / (1 + violations)

# Chooose parents crossover

In [29]:
def select_parents(population):
    # chon nua ca the toi nhat tu featness
    return sorted(population, key=lambda x: fitness(x), reverse=True)[:POPULATION_SIZE // 2]

# Crossover

In [30]:
# lai ghep
def crossover(parent1, parent2):
    child = parent1.copy()
    for period in range(NUM_PERIODS):
        for day in range(NUM_DAYS):
            for week in range(NUM_WEEKS):
                if random.random() < 0.5:
                    child[period, day, week] = parent2[period, day, week]
    return child

# Mutate

In [31]:
# dot bien
def mutate(child, mutation_rate):
    if random.random() < mutation_rate:
        period = random.randint(0, NUM_PERIODS - 1)
        day = random.randint(0, NUM_DAYS - 1)
        week = random.randint(0, NUM_WEEKS - 1)
        room_idx = random.randint(0, NUM_ROOMS - 1)
        subject = random.choice(subjects)
        subject_code = str(subject['Mã môn học'])
        subject_name = subject['Tên môn học']
        teacher_candidates = [t for t, subjs in teacher_subjects.items() if subject_code in subjs]
        if teacher_candidates:
            teacher = random.choice(teacher_candidates)
            room = rooms[room_idx]
            if room['Hình thức'] == subject['Hình thức']:
                child[period, day, week, room_idx] = f"{subject_code},{subject_name},{teacher}"


# Performance evaluation

In [32]:
def track_progress(current_best_fitness, best_fitness, stagnant_generations):
    if current_best_fitness > best_fitness:
        # Reset số thế hệ không cải tiến
        return current_best_fitness, 0  
    else:
        return best_fitness, stagnant_generations + 1

# Evolve

In [33]:
def evolve_population(population, num_generations, mutation_rate):
    # Fitness tốt nhất ban đầu
    best_fitness = 0  
    # Đếm số thế hệ không cải tiến
    stagnant_generations = 0  
    MAX_STAGNANT_GENERATIONS = 500

    for generation in range(num_generations):
        # Chọn lọc
        new_population = select_parents(population)

        # Lai ghép và đột biến 
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.choices(new_population, k=2)
            child = crossover(parent1, parent2)
            mutate(child, mutation_rate)
            new_population.append(child)

        # Cập nhật 
        population = new_population

        # theo doi 
        current_best_fitness = fitness(population[0])
        best_fitness, stagnant_generations = track_progress(current_best_fitness, best_fitness, stagnant_generations)
        print(f"Generation {generation + 1}: Best Fitness = {current_best_fitness}")

        # Dừng khi ko cai thien
        if stagnant_generations >= MAX_STAGNANT_GENERATIONS:
            print(f"No improvement in {MAX_STAGNANT_GENERATIONS} generations. Stopping early.")
            break
    return max(population, key=fitness)

# Run 

In [34]:
# khoi tao
population = [generate_chromosome() for _ in range(POPULATION_SIZE)]

In [35]:
#  tien hoa
best_schedule = evolve_population(population, NUM_GENERATIONS, MUTATION_RATE)

Generation 1: Best Fitness = 0.0018832391713747645
Generation 2: Best Fitness = 0.0035587188612099642
Generation 3: Best Fitness = 0.0035587188612099642
Generation 4: Best Fitness = 0.0038314176245210726
Generation 5: Best Fitness = 0.005235602094240838
Generation 6: Best Fitness = 0.007633587786259542
Generation 7: Best Fitness = 0.012345679012345678
Generation 8: Best Fitness = 0.012345679012345678
Generation 9: Best Fitness = 0.01639344262295082
Generation 10: Best Fitness = 0.01639344262295082
Generation 11: Best Fitness = 0.03225806451612903
Generation 12: Best Fitness = 0.03225806451612903
Generation 13: Best Fitness = 0.047619047619047616
Generation 14: Best Fitness = 0.047619047619047616
Generation 15: Best Fitness = 0.047619047619047616
Generation 16: Best Fitness = 1.0
Generation 17: Best Fitness = 1.0
Generation 18: Best Fitness = 1.0
Generation 19: Best Fitness = 1.0
Generation 20: Best Fitness = 1.0
Generation 21: Best Fitness = 1.0
Generation 22: Best Fitness = 1.0
Genera

# Take the population with the best results

In [36]:
# trich xuat
schedule_data = []
for week in range(NUM_WEEKS):
    for day in range(NUM_DAYS):
        for period in range(NUM_PERIODS):
            for room_idx in range(NUM_ROOMS):
                entry = best_schedule[period, day, week, room_idx]
                if entry:
                    subject_code, subject_name, teacher = entry.split(',')
                    schedule_data.append({
                        "Week": week + 1,
                        "Day": day + 2,
                        "Period": period + 1,
                        "Room": rooms[room_idx]['Tên Phòng'],
                        "Subject": subject_name.strip(),
                        "Teacher": teacher.strip()
                    })

In [37]:
# Tao df
df_schedule = pd.DataFrame(schedule_data)
print(df_schedule.head()) 

   Week  Day  Period    Room                           Subject  \
0     1    2       1  A1.01A                 Hệ thống Máy tính   
1     1    2       3   A1.03             Kỹ năng làm việc nhóm   
2     1    2       3   A2.02                Nhập môn Lập trình   
3     1    2       5   A1.03  Giáo dục Quốc phòng và An Ninh 1   
4     1    2       5   A2.01             Nhập môn Lập trình_TH   

            Teacher  
0   Nguyễn Hữu Tình  
1   Nguyễn Chí Kiên  
2    Bùi Thanh Hùng  
3  Huỳnh Trung Hiếu  
4    Bùi Thanh Hùng  


# Store the results after processing

In [38]:
# Xuất ra file Excel
df_schedule.to_excel("optimal_schedule.xlsx", index=False)
print("Lịch trình đã được lưu vào file 'optimal_schedule.xlsx'.")

Lịch trình đã được lưu vào file 'optimal_schedule.xlsx'.


In [39]:
df_schedule['Room']

0    A1.01A
1     A1.03
2     A2.02
3     A1.03
4     A2.01
5     A1.04
6     A1.05
7     A2.02
8     A2.04
9     A1.05
Name: Room, dtype: object

In [40]:
index=[f"Period {i}" for i in range(1, NUM_PERIODS + 1)]
index

['Period 1', 'Period 2', 'Period 3', 'Period 4', 'Period 5']

In [41]:
columns=[f"Day {i}" for i in range(2, 8)]
columns 

['Day 2', 'Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7']

In [42]:
timetable = pd.DataFrame(index=[f"Period {i}" for i in range(1, NUM_PERIODS + 1)],
                             columns=[f"Day {i}" for i in range(2, 8)])
timetable 

,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
Period 1,NaN,NaN,NaN,NaN,NaN,NaN
Period 2,NaN,NaN,NaN,NaN,NaN,NaN
Period 3,NaN,NaN,NaN,NaN,NaN,NaN
Period 4,NaN,NaN,NaN,NaN,NaN,NaN
Period 5,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
for room in df_schedule['Room'].unique():
    print(room)

A1.01A
A1.03
A2.02
A2.01
A1.04
A1.05
A2.04


In [44]:
for room in df_schedule['Room'].unique():
    room_schedule = df_schedule[df_schedule['Room'] == room]
    for _, row in room_schedule.iterrows():
        print(row)

Week                       1
Day                        2
Period                     1
Room                  A1.01A
Subject    Hệ thống Máy tính
Teacher      Nguyễn Hữu Tình
Name: 0, dtype: object
Week                           1
Day                            2
Period                         3
Room                       A1.03
Subject    Kỹ năng làm việc nhóm
Teacher          Nguyễn Chí Kiên
Name: 1, dtype: object
Week                                      1
Day                                       2
Period                                    5
Room                                  A1.03
Subject    Giáo dục Quốc phòng và An Ninh 1
Teacher                    Huỳnh Trung Hiếu
Name: 3, dtype: object
Week                        1
Day                         2
Period                      3
Room                    A2.02
Subject    Nhập môn Lập trình
Teacher        Bùi Thanh Hùng
Name: 2, dtype: object
Week                    1
Day                     8
Period                  1
Room          

In [45]:
for room in df_schedule['Room'].unique():
    room_schedule = df_schedule[df_schedule['Room'] == room]
    for _, row in room_schedule.iterrows():
        print(f"Period {row['Period']}")
        print(f"Day {row['Day']}")
        print(f"{row['Subject']} ({row['Teacher']})")

Period 1
Day 2
Hệ thống Máy tính (Nguyễn Hữu Tình)
Period 3
Day 2
Kỹ năng làm việc nhóm (Nguyễn Chí Kiên)
Period 5
Day 2
Giáo dục Quốc phòng và An Ninh 1 (Huỳnh Trung Hiếu)
Period 3
Day 2
Nhập môn Lập trình (Bùi Thanh Hùng)
Period 1
Day 8
Toán cao cấp 1 (Lê Trọng Ngọc)
Period 5
Day 2
Nhập môn Lập trình_TH (Bùi Thanh Hùng)
Period 2
Day 4
Kỹ thuật lập trình_TH (Bùi Thanh Hùng)
Period 1
Day 8
Kỹ thuật lập trình (Bùi Thanh Hùng)
Period 2
Day 8
Toán cao cấp 1 (Lê Trọng Ngọc)
Period 1
Day 8
Hệ thống Máy tính_TH (Nguyễn Hữu Tình)


In [47]:
import os

# Tạo thư mục 'images' nếu chưa tồn tại
output_dir = "images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for room in df_schedule['Room'].unique():
    room_schedule = df_schedule[df_schedule['Room'] == room]
    timetable = pd.DataFrame(
        index=[f"Period {i}" for i in range(1, NUM_PERIODS + 1)],
        columns=[f"Day {i}" for i in range(2, 8)]  # Thứ 2 đến Chủ nhật
    )

    for _, row in room_schedule.iterrows():
        period = f"Period {row['Period']}"
        day = f"Day {row['Day']}"
        info = f"{row['Subject']} ({row['Teacher']})"
        timetable.loc[period, day] = info

    timetable.fillna("", inplace=True)

    fig, ax = plt.subplots(figsize=(40, 24))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(
        cellText=timetable.values, 
        colLabels=timetable.columns,
        rowLabels=timetable.index, 
        cellLoc='center', 
        loc='center'
    )
    table.set_fontsize(10)
    table.scale(1, 1)

    plt.title(f"Thời khóa biểu cho phòng: {room}", fontsize=20)

    # Lưu hình ảnh vào thư mục 'images'
    output_file = os.path.join(output_dir, f"{room}_schedule_fixed_v2.png")
    plt.savefig(output_file)
    plt.close()

print("Các bảng thời khóa biểu đã được điều chỉnh và lưu vào thư mục 'images'.")


Các bảng thời khóa biểu đã được điều chỉnh và lưu vào thư mục 'images'.
